In [7]:
import pymysql

In [164]:
#Connect Python to Yelp database in Mysql

import pandas as pd
db = pymysql.connect("localhost","root","qyt007","Yelp")
df_b = pd.read_sql('SELECT * FROM business', db)
df_r = pd.read_sql('SELECT * FROM review', db)
df_u = pd.read_sql('SELECT * FROM user', db)
df_t = pd.read_sql('SELECT * FROM tip', db)
df_c = pd.read_sql('SELECT * FROM checkin', db)


In [165]:
#The business table

df_b.head()

,business_id,city,neighborhood,name,longitude,state,postal_code,categories,stars,address,latitude,review_count,is_open
0,Apn5Q_b6Nz61Tq4XzPdf9A,Calgary,,Minhas Micro Brewery,-114.031675,AB,T2E 6L6,Tours Breweries Pizza Restaurants Food Hotels ...,4.0,1314 44 Avenue NE,51.091813,24,1
1,AjEbIBw6ZFfln7ePHha9PA,Henderson,,CK'S BBQ & Catering,-114.939821,NV,89002,Chicken Wings Burgers Caterers Street Vendors ...,4.5,,35.960734,3,0
2,O8S5hYJ1SMc8fA4QBtVujA,Montral,Rosemont-La Petite-Patrie,La Bastringue,-73.599300,QC,H2G 1K7,Breakfast & Brunch Restaurants French Sandwich...,4.0,1335 rue Beaubien E,45.540503,5,0
3,bFzdJJ3wp3PZssNEsyU23g,Phoenix,,Geico Insurance,-112.076979,AZ,85003,Insurance Financial Services,1.5,211 W Monroe St,33.449999,8,1
4,8USyCYqpScwiNEb58Bt6CA,Calgary,,Action Engine,-114.027366,AB,T2H 0N5,Home & Garden Nurseries & Gardening Shopping L...,2.0,2005 Alyth Place SE,51.035591,4,1


In [162]:
#The review table

df_r.head()

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
0,0,msQe1u7Z_XuqjGoqhB0J5g,x7mDIiDB3jEiPGPHOmDzyw,The pizza was okay. Not the best I've had. I p...,iCQpiavjjPzJ5_3gPD5Ebg,2,2011-02-25,0,0
1,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0
2,1,msQe1u7Z_XuqjGoqhB0J5g,LZp4UX5zK3e-c5ZGSeo3kA,Terrible. Dry corn bread. Rib tips were all fa...,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,3,1
3,0,msQe1u7Z_XuqjGoqhB0J5g,Er4NBWCmCD4nM8_p1GRdow,Back in 2005-2007 this place was my FAVORITE t...,elqbBhBfElMNSrjFqW3now,2,2011-02-25,2,0
4,0,msQe1u7Z_XuqjGoqhB0J5g,jsDu6QEJHbwP2Blom1PLCA,Delicious healthy food. The steak is amazing. ...,Ums3gaP2qM3W1XcA5r6SsQ,5,2014-09-05,0,0


In [98]:
#To confirm the data type of 'date' attribute

type(df_r.iloc[0,6])

datetime.date

In [9]:
#Join review and business dataframes on 'business_id'

df_r_b = pd.merge(df_r, df_b, on ='business_id')

In [17]:
#3. How many Canadian residents reviewed the business “Mon Ami Gabi” in last 1 year?

#3. Records of people who have visited 'Mon Ami Gabi' in last 1 year

import datetime
df_r_MAG = df_r_b[(df_r_b['name']=='Mon Ami Gabi')&(df_r_b['date']> datetime.date(2017, 8, 8))]

In [214]:
#3. Records of reviews and business locaed in Canada

df_CA = df_r_b[df_r_b['postal_code'].str.contains('[A-Z]')]

In [217]:
#3. Since there's no nationality information in user table
#3. It's very hard to tell whether a user is a Canadian resident 
#3. First, I tried to calculate the number of people who reviewed the business “Mon Ami Gabi” in last 1 year and also reviewed business located in Canada
#3. But obviously, not all of these 44 users are Canadian residents
#3. Maybe some of them traveled to Canada and reviewed the business

unique_id = set(df_r_MAG['user_id'][df_r_MAG['user_id'].isin(df_CA['user_id'])])
unique_id_count = len(unique_id)
unique_id_count

44

In [247]:
#3. Thus, I decided to identify users based on how many times they've reviewed business in Canada
#3. For users who only reviewed business in Canada once
#3. I'll say they're not Canada residents

df_3 = df_r_b[(df_r_b['user_id'].isin(df_r_MAG['user_id'])) & (df_r_b['user_id'].isin(df_CA['user_id']))]
df_pos = df_3[['user_id','postal_code']]
s_CA = df_pos[df_pos['postal_code'].str.contains('[A-Z]')].groupby('user_id')['postal_code'].count()
s_CA1 = s_CA[s_CA==1]
s_CA1.count()

4

In [258]:
#3. For users who reviewed business in Canada a few times
#3. It's hard to say, but I guess there' a probability of 50% that they are Canada residents
#3. So let's give them a weight of 50%

s_CA2 = s_CA[(s_CA>1) & (s_CA<=10)]
CA2_num = s_CA2.count()*0.5
CA2_num

12.0

In [256]:
#3. For users who reviewed business in Canada many times
#3. I guess there' a probability of 80% that they are Canada residents
#3. So let's give them a weight of 80%

s_CA3 = s_CA[(s_CA>10) & (s_CA<=100)]
CA3_num = s_CA3.count()*0.8
CA3_num 

11.200000000000001

In [254]:
#3. For users who reviewed business in Canada more than 100 times
#3. I'll say they are Canada residents
#3. So let's give them a weight of 70%

s_CA4 = s_CA[s_CA>100]
CA4_num = s_CA4.count()
CA4_num

2

In [259]:
#3. Let's add up and see approximately how many Canadian residents reviewed 'Mon Ami Gabi' in last 1 year

CA_num = CA2_num + CA3_num + CA4_num
CA_num

25.200000000000003

In [168]:
#4. Top 10 most common words in the reviews of the business “Chipotle Mexican Grill”

#4. Read the joint dataframe, and noticed there were duplicates

df_r_b.head()

,funny,user_id,review_id,text,business_id,stars_x,date,useful,cool,city,...,name,longitude,state,postal_code,categories,stars_y,address,latitude,review_count,is_open
0,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
1,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
2,0,xJU_8kB-QVdK-_BOotvi9Q,wkpqXCNE3heeD-uexZ1NaA,My fiance and I decided to check this place ou...,pomGBqfbxcqPv14c3XH-ZQ,2,2016-11-04,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
3,0,cdFB_a_ZXF7qGVKP2mYikA,FXt7F4e-ncrq5pYstUVwxw,"""We were really looking forward to liking this...",pomGBqfbxcqPv14c3XH-ZQ,1,2011-08-10,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
4,0,ZdUFyF5eNyZxete-T4bmZA,gzG_aKOfpspNELvX4Msy6w,"""This was our """"go to"""" Mexican spot for the l...",pomGBqfbxcqPv14c3XH-ZQ,2,2014-06-04,4,1,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1


In [47]:
#4. Check the user_id and find that there are duplicates for all of his/her reviews

df_r_b[df_r_b['user_id'] == 'msQe1u7Z_XuqjGoqhB0J5g']

,funny,user_id,review_id,text,business_id,stars_x,date,useful,cool,city,...,name,longitude,state,postal_code,categories,stars_y,address,latitude,review_count,is_open
0,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
1,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
1126,1,msQe1u7Z_XuqjGoqhB0J5g,LZp4UX5zK3e-c5ZGSeo3kA,Terrible. Dry corn bread. Rib tips were all fa...,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,3,1,Las Vegas,...,H&H BBQ Plus 2,-115.234495,NV,89130,American (New) Barbeque Restaurants,3.5,4510 N Rancho Dr,36.241809,115,1
1127,1,msQe1u7Z_XuqjGoqhB0J5g,LZp4UX5zK3e-c5ZGSeo3kA,Terrible. Dry corn bread. Rib tips were all fa...,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,3,1,Las Vegas,...,H&H BBQ Plus 2,-115.234495,NV,89130,American (New) Barbeque Restaurants,3.5,4510 N Rancho Dr,36.241809,115,1
1242,0,msQe1u7Z_XuqjGoqhB0J5g,Er4NBWCmCD4nM8_p1GRdow,Back in 2005-2007 this place was my FAVORITE t...,elqbBhBfElMNSrjFqW3now,2,2011-02-25,2,0,Las Vegas,...,Pin Kaow Thai Restaurant,-115.241323,NV,89108,Restaurants Thai,4.0,1974 N Rainbow Blvd,36.195393,601,1
1243,0,msQe1u7Z_XuqjGoqhB0J5g,Er4NBWCmCD4nM8_p1GRdow,"""Back in 2005-2007 this place was my FAVORITE ...",elqbBhBfElMNSrjFqW3now,2,2011-02-25,2,0,Las Vegas,...,Pin Kaow Thai Restaurant,-115.241323,NV,89108,Restaurants Thai,4.0,1974 N Rainbow Blvd,36.195393,601,1
1844,0,msQe1u7Z_XuqjGoqhB0J5g,jsDu6QEJHbwP2Blom1PLCA,Delicious healthy food. The steak is amazing. ...,Ums3gaP2qM3W1XcA5r6SsQ,5,2014-09-05,0,0,Las Vegas,...,Braddah's Island Style,-115.287045,NV,89149,Restaurants Desserts Asian Fusion Mexican Hawa...,4.0,6410 N Durango Dr,36.277614,241,0
1845,0,msQe1u7Z_XuqjGoqhB0J5g,jsDu6QEJHbwP2Blom1PLCA,Delicious healthy food. The steak is amazing. ...,Ums3gaP2qM3W1XcA5r6SsQ,5,2014-09-05,0,0,Las Vegas,...,Braddah's Island Style,-115.287045,NV,89149,Restaurants Desserts Asian Fusion Mexican Hawa...,4.0,6410 N Durango Dr,36.277614,241,0
2086,0,msQe1u7Z_XuqjGoqhB0J5g,pfavA0hr3nyqO61oupj-lA,This place sucks. The customer service is horr...,vgfcTvK81oD4r50NMjU2Ag,1,2011-02-25,2,0,Las Vegas,...,Larry's Hideaway,-115.212560,NV,89130,Bars Nightlife Dive Bars,2.5,3369 Thom Blvd,36.220060,8,0
2087,0,msQe1u7Z_XuqjGoqhB0J5g,pfavA0hr3nyqO61oupj-lA,"""This place sucks. The customer service is hor...",vgfcTvK81oD4r50NMjU2Ag,1,2011-02-25,2,0,Las Vegas,...,Larry's Hideaway,-115.212560,NV,89130,Bars Nightlife Dive Bars,2.5,3369 Thom Blvd,36.220060,8,0


In [176]:
#4. Drop all the duplicate reviews

df_r_b.drop_duplicates('review_id',keep='first',inplace=True)
df_r_b.head()

,funny,user_id,review_id,text,business_id,stars_x,date,useful,cool,city,...,name,longitude,state,postal_code,categories,stars_y,address,latitude,review_count,is_open
0,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
2,0,xJU_8kB-QVdK-_BOotvi9Q,wkpqXCNE3heeD-uexZ1NaA,My fiance and I decided to check this place ou...,pomGBqfbxcqPv14c3XH-ZQ,2,2016-11-04,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
3,0,cdFB_a_ZXF7qGVKP2mYikA,FXt7F4e-ncrq5pYstUVwxw,"""We were really looking forward to liking this...",pomGBqfbxcqPv14c3XH-ZQ,1,2011-08-10,0,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
4,0,ZdUFyF5eNyZxete-T4bmZA,gzG_aKOfpspNELvX4Msy6w,"""This was our """"go to"""" Mexican spot for the l...",pomGBqfbxcqPv14c3XH-ZQ,2,2014-06-04,4,1,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1
5,0,jEBySjq6tgL-_R3P7LHq0w,avx1RLsFHu8Z5M4HVWBRzg,If you love Mexican food I totally recommend t...,pomGBqfbxcqPv14c3XH-ZQ,4,2011-08-10,2,0,Las Vegas,...,Leticia's Mexican Cocina,-115.280088,NV,89143,Restaurants Mexican Bars Nightlife,4.0,7585 Norman Rockwell Ln Bldg 1,36.298875,1125,1


In [76]:
#4. Top 10 most common words in the reviews of the business “Chipotle Mexican Grill”

from itertools import chain
from collections import Counter
from nltk.tokenize import wordpunct_tokenize
print Counter(chain.from_iterable(wordpunct_tokenize(x) for x in df_r_b['text'][df_r_b['name']=='Chipotle Mexican Grill'])).most_common(10)

[('.', 42782), ('the', 29954), ('I', 27030), ('and', 22619), ('to', 19643), ('a', 16414), ("'", 15991), ('is', 11151), ('of', 10587), ('it', 9907)]


In [210]:
#5. What’s the percentage of users, who reviewed “Mon Ami Gabi”, and also reviewed at least 10 other restaurants located in Ontario?

#5. Find users who reviewed at least 10 restaurants in ON

df_ON = df_r_b[['user_id','review_id']][df_r_b['state']=='ON']
s_ON = df_ON.groupby(['user_id'])['review_id'].count()
s_10ON = s_ON[s_ON>=10]
s_10ON.head(10)

user_id
--BumyUHiO_7YsHurb9Hkw     43
--Qh8yKWAvIP4V4K8ZPfHA    378
--UOvCH5qEgdNQ8lzR8QYQ     10
--YhjyV-ce1nFLYxP49C5A     82
-0AyZxS5C--WySnbW_Q8yQ     18
-0kf3OEgFh6APiSedF4kbg     13
-1MF2tosrw2WcCxeVNk81Q     10
-1aYNZASC1lkA09LJtfNAA     13
-1l27u1nHhe7F7AlheMIGQ     13
-1wbglcr6x1qrUbqP1YAIA     30
Name: review_id, dtype: int64

In [213]:
#5. Find users who reviewed reviewed 'Mon Ami Gabi'
#5. and calculate the percentage

count_MAG = df_r_b['user_id'][df_r_b['name']=='Mon Ami Gabi'].count()
s = df_r_b['user_id'][df_r_b['name']=='Mon Ami Gabi'].isin(s_10ON.index)
count_ON10 = s[s == True].count()
print('%.2f%%'%(float(count_ON10)/float(count_MAG)*100))

0.89%


In [175]:
#6. Analytics1: Which states should I start or expand my business in the US and in Canada?

df_b['popularity'] = df_b['stars']*df_b['review_count']
df_b.head()

,business_id,city,neighborhood,name,longitude,state,postal_code,categories,stars,address,latitude,review_count,is_open,popularity
0,Apn5Q_b6Nz61Tq4XzPdf9A,Calgary,,Minhas Micro Brewery,-114.031675,AB,T2E 6L6,Tours Breweries Pizza Restaurants Food Hotels ...,4.0,1314 44 Avenue NE,51.091813,24,1,96.0
1,AjEbIBw6ZFfln7ePHha9PA,Henderson,,CK'S BBQ & Catering,-114.939821,NV,89002,Chicken Wings Burgers Caterers Street Vendors ...,4.5,,35.960734,3,0,13.5
2,O8S5hYJ1SMc8fA4QBtVujA,Montral,Rosemont-La Petite-Patrie,La Bastringue,-73.599300,QC,H2G 1K7,Breakfast & Brunch Restaurants French Sandwich...,4.0,1335 rue Beaubien E,45.540503,5,0,20.0
3,bFzdJJ3wp3PZssNEsyU23g,Phoenix,,Geico Insurance,-112.076979,AZ,85003,Insurance Financial Services,1.5,211 W Monroe St,33.449999,8,1,12.0
4,8USyCYqpScwiNEb58Bt6CA,Calgary,,Action Engine,-114.027366,AB,T2H 0N5,Home & Garden Nurseries & Gardening Shopping L...,2.0,2005 Alyth Place SE,51.035591,4,1,8.0


In [180]:
#6. Analytics1: Top 20 popular states in US and Canada 
#6. Analytics1: If you are in the US, you'd better start or expand your business in NV, AZ, and NC.
#6. Analytics1: If you are in Canada, you'd better start or expand your business in ON and QC. 

s_states = df_b.groupby(['state'])['popularity'].mean()
s_states.sort_values(ascending = False).head(20)

state
NV     210.288335
AZ     126.519349
NC      93.903750
WI      90.516363
PA      89.406200
ON      77.719054
IL      73.791973
OH      73.137492
QC      71.281985
FL      68.000000
HH      67.500000
SC      61.595142
OR      61.294118
WA      60.500000
TAM     56.000000
MA      49.000000
AR      47.500000
IN      45.734043
SP      45.000000
AB      42.816174
Name: popularity, dtype: float64

In [181]:
#6. Analytics2: Which neighborhood should I start or expand my business in the US and in Canada?

df_sorted = df_b.sort_values(by = ['popularity'],ascending = False)
df_sorted_50 = df_sorted.head(50)
df_sorted_50.head()

,business_id,city,neighborhood,name,longitude,state,postal_code,categories,stars,address,latitude,review_count,is_open,popularity
137630,4JNXUYY8wbaaDmk3BPzlWw,Las Vegas,The Strip,Mon Ami Gabi,-115.172582,NV,89109,Steakhouses Breakfast & Brunch Restaurants French,4.0,3655 Las Vegas Blvd S,36.112827,7968,1,31872.0
62720,K7lWdNUhCbcnEvI0NhGewg,Las Vegas,The Strip,Wicked Spoon,-115.176155,NV,89109,Buffets Restaurants Breakfast & Brunch,3.5,3708 Las Vegas Blvd S,36.109550,6446,1,22561.0
177567,DkYS3arLOhA8si5uUEmHOw,Las Vegas,The Strip,Earl of Sandwich,-115.171869,NV,89109,Sandwiches Wraps Food Caterers Restaurants Sou...,4.5,3667 Las Vegas Blvd S,36.109443,4981,1,22414.5
170123,f4x1YBxkLrZg652xt2KR5g,Las Vegas,The Strip,Hash House A Go Go,-115.171580,NV,89109,Breakfast & Brunch American (New) Restaurants,4.0,3535 Las Vegas Blvd,36.118181,5382,1,21528.0
96847,hihud--QRriCYZw1zZvW4g,Las Vegas,Eastside,Gangnam Asian BBQ Dining,-115.151261,NV,89169,Asian Fusion Barbeque Korean Japanese Restaura...,4.5,4480 Paradise Rd Ste 600,36.108898,3731,1,16789.5


In [182]:
#6. Analytics2: You'd better start or expand your business in Eastside, Downtown, and the Strip.

s_neighborhood = df_sorted_50.groupby(['neighborhood'])['popularity'].mean().sort_values(ascending=False)
s_neighborhood

neighborhood
Eastside         16789.500000
Downtown         15900.000000
The Strip        13324.155172
Southeast        12591.750000
Westside         11874.333333
Chinatown         9985.500000
                  9388.454545
Spring Valley     9288.000000
Name: popularity, dtype: float64

In [93]:
#6. Analytics3: Which categories should I start or expand my business?
#6. Analytics3: You'd better start or expand your business in categories such as 'Restaurants','American',
#6. 'Hotels','Services','Event','Bars','Food','Nightlife'...

print Counter(chain.from_iterable(wordpunct_tokenize(x) for x in df_sorted_50['categories'])).most_common(15)

[('&', 50), ('Restaurants', 40), ('(', 28), ('American', 28), (')', 28), ('New', 19), ('Hotels', 19), ('Services', 14), ('Event', 14), ('Bars', 14), ('Food', 14), ('Nightlife', 13), ('Planning', 12), ('Breakfast', 12), ('Brunch', 12)]


In [183]:
#Section 2.2
#predicting stars based on review texts
#get star list

stars = df_r['stars'].tolist()

In [185]:
#difficulty1: different users may have different standards, similar reviews may result in different star ratings
#difficulty2: the dataset is unbalanced, we have more texts that typically have 5-star rating than texts that typically have 2-star rating. But according to the base of most of the machine learning classifiers, we need balanced numbers of reviews of each star rating type.

print(Counter(stars))

Counter({5: 2644595, 4: 1337162, 1: 859164, 3: 673818, 2: 488340, 0: 1})


In [145]:
#Balance the data first

df_new = df_r.copy()   
max_num = 488340
df_5 = df_new[df_new['stars']==5].head(488340)
df_4 = df_new[df_new['stars']==4].head(488340)
df_3 = df_new[df_new['stars']==3].head(488340)
df_2 = df_new[df_new['stars']==2].head(488340)
df_1 = df_new[df_new['stars']==1].head(488340)
df_balanced = pd.concat([df_5,df_4,df_3,df_2,df_1])

In [187]:
df_balanced.head()

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
1,0,msQe1u7Z_XuqjGoqhB0J5g,dDl8zu1vWPdKGihJrwQbpw,I love this place! My fiance And I go here atl...,pomGBqfbxcqPv14c3XH-ZQ,5,2012-11-13,0,0
4,0,msQe1u7Z_XuqjGoqhB0J5g,jsDu6QEJHbwP2Blom1PLCA,Delicious healthy food. The steak is amazing. ...,Ums3gaP2qM3W1XcA5r6SsQ,5,2014-09-05,0,0
6,0,msQe1u7Z_XuqjGoqhB0J5g,brokEno2n7s4vrwmmUdr9w,If you like Thai food you have to try the orig...,AxeQEz3-s9_1TyIo-G7UQw,5,2011-10-10,1,0
7,1,msQe1u7Z_XuqjGoqhB0J5g,kUZWBVZvhWuC8TWUg5AYyA,AMAZING!!! I was referred here by a friend and...,zdE82PiD6wquvjYLyhOJNA,5,2012-04-18,0,0
10,0,TlvV-xJhmh7LCwJYXkV-cg,STiFMww2z31siPY7BWNC2g,I have been an Emerald Club member for a numbe...,yFumR3CWzpfvTH2FCthvVw,5,2016-06-15,0,0


In [147]:
#Data has been balanced

Counter(df_balanced['stars'].tolist())

Counter({1: 488340, 2: 488340, 3: 488340, 4: 488340, 5: 488340})

In [148]:
#Text data vectorization
#The vectorizer breaks text into single words and bi-grams, and then calculates the TF-IDF representation

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1,2))
vectors = vectorizer.fit_transform(df_balanced['text'])

In [149]:
#Train-test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, df_balanced['stars'], test_size = 0.33, random_state = 42)

In [150]:
#Fitting a classifier

from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [153]:
#Use the fitted model to make predictions 
#looked at the predictions for the first 20 reviews in the test set

prediction = classifier.predict(X_test)
print(list(prediction[:20]))
print(list(y_test[:20]))

[2, 5, 5, 1, 4, 5, 3, 4, 4, 5, 4, 4, 2, 2, 5, 2, 1, 1, 5, 4]
[2, 5, 5, 2, 4, 4, 3, 4, 4, 5, 4, 4, 1, 2, 5, 3, 2, 1, 4, 4]


In [155]:
#Sanity check
#I'll take a quick look at the mis-categorized reviews, and try to predict a star by myself

print(df_balanced['text'][3])

Back in 2005-2007 this place was my FAVORITE thai place EVER. I'd go here ALLLLL the time. I never had any complaints. Once they started to get more known and got busy their service started to suck and their portion sizes got cut in half. I have a huge problem with paying MORE for way less food. The last time I went there I had the Pork Pad se Ew and it tasted good but I finished my plate and was still hungry. I used to know the manager here and she would greet me with a "Hello Melissa nice to see you again diet coke & pad thai or pad se ew?"" Now a days I know she still knows me but she disregards my presence. Also I had asked her what was up with the new portion sizes and she had no answer for me. Great food but not worth the money. I havent been back in over a year because I refuse to pay $10-15 for dinner and still be hungry after. Sorry PinKaow you are not what you used to be!!"


In [156]:
#Evaluate the performance of the classifier using accuracy score

from sklearn.metrics import accuracy_score
accuracy_score(y_test,prediction)

0.6329172049776547

In [158]:
#Evaluate the performance of the classifier using precision(false-negative) and recall(false-positive)

from sklearn.metrics import classification_report
print(classification_report(y_test,prediction))

             precision    recall  f1-score   support

          1       0.73      0.79      0.76    161146
          2       0.56      0.53      0.54    161528
          3       0.55      0.51      0.53    161014
          4       0.57      0.56      0.56    161157
          5       0.74      0.78      0.76    160916

avg / total       0.63      0.63      0.63    805761



In [160]:
# Use confusion matrix for evaluating which predictions are often confused

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,prediction))

[[126980  28141   4204    896    925]
 [ 37384  84836  33316   4579   1413]
 [  7625  33903  82400  32333   4753]
 [  1596   4642  27566  90151  37202]
 [  1122    782   3093  30306 125613]]
